# Si446x Device Direct Access TagNet Poll

In [ ]:
!pwd
%autosave 0
import sys
sys.path.append("../si446x/si446x")
%run '../si446x/si446x/notebooks/si446x_Device_Layer.ipynb'

In [ ]:
import sys
sys.path.append("../tagnet/tagnet")
from tagmessages import TagMessage, TagPoll, TagGet
from tagnames import TagName
from tagtlv import TagTlv, tlv_types
tm = TagMessage(TagName('/tag/poll'))

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = si446x_device_start_radio()

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

##  Configure Radio

In [ ]:
config = si446x_device_config_radio(radio)

si446x_device_show_config(radio.dump_radio())
total = 0
print('\n=== const config strings:')
for s in config:
    print((hexlify(s)))
    total += len(s) - 4
print('\n total: {}'.format(total))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

## Map position calculation

In [ ]:
import pyproj
import struct

In [ ]:
# Example position data, should be somewhere in Germany
x = 652954.1006
y = 4774619.7919
z = -2217647.7937

ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

def conv_xyz_to_lat(x, y, z):
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z, radians=True)
    return lat, lon, alt

def payload_to_lat(byte_range):
    lat, lon, alt = conv_xyz_to_lat(struct.unpack('>LLL', bytearray(byte_range)))
    return lat, lon, alt

## Transmit TagNet Poll Request

In [ ]:
from binascii import hexlify
tp = TagPoll()
print(tp.name)
print(tp.payload)
poll_msg = tp.build()
print(len(poll_msg),hexlify(poll_msg))

In [ ]:
xn = TagName('/tag/info/') + TagTlv(tlv_types.NODE_ID, -1)\
         + TagTlv('sens') + TagTlv('gps') + TagTlv('xyz')
xg = TagGet(xn)
xyz_msg = xg.build()
print(xg.name)
print(xg.header)
print(len(xyz_msg),hexlify(xyz_msg))

In [ ]:
from __future__ import print_function
from time import sleep
from datetime import datetime


num         = 1
MAX_WAIT    = 20
MAX_RECV    = 256

mj_r        = range(num)
th_r        = range(40,41,5)              # fifo threshold setting (min,max,opt): 1,64,40
pw_r        = [1]
#pw_r        = [1 << i for i in range(2)] # power amp setting (min, max,opt): 1,127,10
delay       = .01

prp         = bytearray(1)

xcnt        = 0
rcnt        = 0
pstr        = ''

rssi        = 0

radio.trace._enable()

prp[0] = 0x22
radio.set_property('MODEM', 0x4c, prp) 

start = datetime.now()
print(start)
req_msg = xyz_msg

for rep in range(num):
    for thresh in th_r:
        prp[0]=thresh
        radio.set_property('PKT', 0x0b, prp)  # set tx fifo threshhold
        for pwr in pw_r:
            if (pwr > 127): pwr = 127
#            req_msg = xyz_msg if (req_msg == poll_msg) else poll_msg
            l = si446x_device_send_msg(radio, req_msg, pwr)
            rmsg, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, MAX_WAIT)
            xcnt += 1
            if (rmsg):
                rcnt += 1
                last_len = len(pstr) if (pstr) else 0
                pstr = '{}, xc/rc:{}/{}, l:{}, t:{}, lp:{}, lr:{}, rsp:{}..{}'.format(
                           datetime.now(), xcnt, rcnt, len(rmsg), thresh,
                           pwr, rssi, hexlify(rmsg[:8]), hexlify(rmsg[-14:]))
                xyz = TagMessage(rmsg)
                print(xyz.header)
                print(xyz.name)
                print(xyz.payload[0].__len__())
                print(len(xyz.payload[0].value()))
                print(payload_to_lat(xyz.payload[0].value()))
            else:
                pstr = '{}, xc/rc:{}/{}, l:{}, t:{}, lp:{}, lr:{}, req:{}'.format(
                            datetime.now(), xcnt, rcnt, len(req_msg), thresh,
                            pwr, rssi, hexlify(req_msg[:8]))
            print(pstr, end='')
            if (status[-1] == 'e'):
                print(', {}'.format(''.join(map(str, status))))
            else:
                x = last_len - len(pstr) if (last_len > len(pstr)) else 1
                print(' ' * x,end='\r')
            sleep(delay)
print('\ndone')

## Check Interrupt Status

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
s_name =  'get_chip_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_chip_status()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
#t,r = si446xradio_test()

In [ ]:
from datetime import datetime 

now=datetime.now()
print(now)

In [ ]:
MAX_CHUNK=64

In [ ]:
radio.clear_interrupts()

In [ ]:
rx, tx = radio.fifo_info(rx_flush=True, tx_flush=True)
print(rx,tx)

In [ ]:
radio.set_property('PKT', 0x0b, b'\x3c')  # set tx fifo threshhold

In [ ]:
rx, tx = radio.fifo_info()
print(rx,tx)

In [ ]:
status = radio.get_interrupts()
print(status.ph_pend.TX_FIFO_ALMOST_EMPTY)

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
buf = bytearray('\x22' * 64)
chunk = 8
cur   = 8
radio.write_tx_fifo(buf[:chunk])
rx, tx = radio.fifo_info()
print(rx,tx)

In [ ]:
radio.start_tx(len(buf))
while (True):
    status = radio.get_interrupts()
    if (status.ph_pend.TX_FIFO_ALMOST_EMPTY):
        rx, tx = radio.fifo_info()
        radio.write_tx_fifo(buf[cur:cur+chunk])
        cur += chunk
        print('.',end='')
    if (status.ph_pend.PACKET_SENT):
        print('done')
        break
    radio.clear_interrupts()

In [ ]:
ss[0] = 10
ss[1] = 20

In [ ]:
cur

In [ ]:
radio.set_property('PKT', 0x0b, ss) # tx/rx threshold

In [ ]:
radio.set_property('PKT', 0x0b, b'\x48')  # set tx fifo threshhold

In [ ]:
radio.set_property('PKT', 0x0c, b'\x14')  # set rx fifo threshold

In [ ]:
status = int_status(clr_all_flags, False)
print(status.ph_pend.TX_FIFO_ALMOST_EMPTY)